In [180]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [181]:
!pip install -r requirements.txt

from src.preprocessing import DataPreprocessor
from src.model import Model, NeuralNetwork, MultiOutputNeuralNetwork
from src.gui import AnalysisGUI

import torch
torch.serialization.add_safe_globals([NeuralNetwork, MultiOutputNeuralNetwork])

You should consider upgrading via the 'C:\Users\csbro\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [182]:
# Load training and testing datasets
dataset = DataPreprocessor(
    filepath="data/Line Listing(dxjZy) (5).xlsx",
    column_names=['Case ID', 'Suspect Product Active Ingredients', 'Serious', 'Sex', 'Patient Age', 'Patient Weight']
)
dataset.explode_column("Suspect Product Active Ingredients", ";")
medications = dataset.data["Suspect Product Active Ingredients"] # SAVED FOR LATER MENUS IN GUI

print("Value Counts:")
print(dataset.get_value_counts("Serious"))

print("\nData Types:")
print(dataset.get_dataframe().dtypes)

dataset.get_dummies(["Suspect Product Active Ingredients"])
dataset.data.groupby("Case ID").max().reset_index()
dataset.drop_columns(["Case ID"])

dataset.convert_nulls("Patient Age", nulls=["Not Specified"], output="0 YR")
dataset.ensure_numeric_column("Patient Age", decimal=True)

dataset.convert_nulls("Patient Weight", nulls=["Not Specified"], output="0 KG")
dataset.ensure_numeric_column("Patient Weight")

dataset.encode_column("Sex")
dataset.encode_column("Serious")

dataset.drop_all_nulls()

print("\nNew Value Counts:")
print(dataset.get_value_counts("Serious"))

print("\nDataset Head:")
print(dataset.get_dataframe().head())

x_train, x_test, y_train, y_test = dataset.get_standardised_train_test_split(
    dataset.data.columns, 
    y="Serious", 
    test_size=0.2, 
    random_state=42
)

Value Counts:
Serious
Serious        21935
Non-Serious    10340
Name: count, dtype: int64

Data Types:
Case ID                                int64
Suspect Product Active Ingredients    object
Serious                               object
Sex                                   object
Patient Age                           object
Patient Weight                        object
dtype: object

New Value Counts:
Serious
1    21935
0    10340
Name: count, dtype: int64

Dataset Head:
    Case ID  Serious  Sex  Patient Age  Patient Weight  .Alpha.-Lipoic Acid  \
0  24795443        1    1         73.0               0                False   
1  24795442        1    0         76.0               0                False   
2  24795440        1    0         67.0               0                False   
3  24792897        0    0         86.0               0                False   
4  24773331        1    1         65.0               0                False   

   .Alpha.-Tocopherol Acetate  Abaloparatide  Ab

In [ ]:
# Loading models
input_dim = len(x_train[0])
serious_model  = Model(input_dim).load_model("serious_model.pth", weights=False)
reaction_model = Model(input_dim).load_model("reaction_model.pth", weights=False)

print(serious_model)

In [184]:
# Show GUI
medications = medications.unique()

gui = AnalysisGUI(
    serious_model=serious_model,
    reaction_model=reaction_model,
    medications=medications
)

TraitError: The 'value' trait of a Label instance expected a unicode string, not the NoneType None.